In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from sklearn.preprocessing import MinMaxScaler

movies = pd.read_csv('/content/drive/MyDrive/BigData/Project/movies_tmdb_5000_cleaned.csv')

print(movies.shape)
movies.head(5)

(5000, 16)


,tmdb_id,title,original_title,release_date,overview,popularity,vote_average,vote_count,original_language,budget,revenue,runtime,genres,keywords,cast,director
0,617126,판타스틱 4: 새로운 출발,The Fantastic 4: First Steps,2025-07-22,"1960년대, 전 세계적인 관심 속 우주로 떠난 4명의 엘리트 우주비행사 ‘리드 리...",642.9939,7.189,1492,en,200000000,520716140,114,"['SF', '모험']","['superhero', 'based on comic', 'end of the wo...","['페드로 파스칼', '바네사 커비', '에번 모스배크랙']",맷 샤크먼
1,1311031,극장판 귀멸의 칼날: 무한성편,劇場版「鬼滅の刃」無限城編 第一章 猗窩座再来,2025-07-18,혈귀로 변해버린 여동생 네즈코를 인간으로 되돌리기 위해 혈귀를 사냥하는 조직인 《귀...,539.9749,7.800,357,ja,20000000,558756781,155,"['애니메이션', '액션', '판타지', '스릴러']","['supernatural', 'sequel', 'based on manga', '...","['하나에 나츠키', '시모노 히로', '사쿠라이 타카히로']",소토자키 하루오
2,755898,우주전쟁,War of the Worlds,2025-07-29,전설적인 동명 소설을 새롭게 재해석한 이번 작품은 거대한 침공의 서막을 알린다. 에...,424.4831,4.400,578,en,0,0,145,"['SF', '스릴러']","['government', 'attack', 'privacy', 'surveilla...","['아이스 큐브', '에바 롱고리아', '클라크 그렉']",Rich Lee
3,987400,Batman Azteca: Choque de imperios,Batman Azteca: Choque de imperios,2025-09-18,"In the time of the Aztec empire, tragedy strik...",294.1757,8.072,69,es,0,0,89,"['애니메이션', '액션', '모험', '판타지']","['death of father', 'based on comic', 'colonis...","['Horacio García Rojas', '알바로 모르테', 'Omar Chap...",Juan Jose Meza-Leon
4,1038392,컨저링: 마지막 의식,The Conjuring: Last Rites,2025-09-03,"1986년 펜실베니아, 자신의 집에 사악한 존재가 들어왔다고 주장하는 스멀 일가를 ...",249.8959,6.600,429,en,55000000,403285328,136,['공포'],"['pennsylvania, usa', 'supernatural', 'exorcis...","['베라 파미가', '패트릭 윌슨', '미아 톰린슨']",마이클 차브스


In [3]:
# 개봉일(release_date) 날짜 형식 변환
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
print(movies['release_date'].dtype)

# 결측치 제거 ??? (처리 어케 할까요 - 하나 있음)
# movies = movies.dropna(subset=['release_date'])
# print(movies.shape)

datetime64[ns]


In [4]:
# 언어(original_language) 결측치 처리 및 소문자화
movies['original_language'] = movies['original_language'].fillna('unknown').str.lower()

In [5]:
# 줄거리(overview) 전처리
def clean_text(text):
  if pd.isna(text):
    return ''
  text = text.lower()
  text = re.sub(r'<[^>]+>', '', text)             # HTML 태그 제거
  text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text) # 특수문자 제거
  text = re.sub(r'\s+', ' ', text).strip()        # 공백 제거
  return text

movies['overview'] = movies['overview'].apply(clean_text)

In [6]:
# 장르(genres), 키워드(keywords) 리스트 변환 및 소문자화
def safe_literal_eval(x):
  if pd.isna(x) or x == '':
    return []
  try:
    return literal_eval(x)
  except:
    return []

def clean_list(lst):
  return [str(item).strip().lower() for item in lst if str(item).strip()]

movies['genres'] = movies['genres'].apply(safe_literal_eval, clean_list)
movies['keywords'] = movies['keywords'].apply(safe_literal_eval, clean_list)

movies[['genres', 'keywords']].head(5)

/tmp/ipython-input-875617598.py:13: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  movies['genres'] = movies['genres'].apply(safe_literal_eval, clean_list)
/tmp/ipython-input-875617598.py:14: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  movies['keywords'] = movies['keywords'].apply(safe_literal_eval, clean_list)


,genres,keywords
0,"[SF, 모험]","[superhero, based on comic, end of the world, ..."
1,"[애니메이션, 액션, 판타지, 스릴러]","[supernatural, sequel, based on manga, demon, ..."
2,"[SF, 스릴러]","[government, attack, privacy, surveillance, cy..."
3,"[애니메이션, 액션, 모험, 판타지]","[death of father, based on comic, colonisation..."
4,[공포],"[pennsylvania, usa, supernatural, exorcism, ha..."


In [7]:
# 장르명 영어로 통일
genre_translate = {
    '코미디': 'comedy',
    '로맨스': 'romance',
    '드라마': 'drama',
    '가족': 'family',
    '음악': 'music',
    '공포': 'horror',
    '판타지': 'fantasy',
    '액션': 'action',
    '모험': 'adventure',
    '애니메이션': 'animation',
    'tv 영화': 'tv movie',
    '범죄': 'crime',
    '스릴러': 'thriller',
    '역사': 'history',
    '미스터리': 'mystery',
    '전쟁': 'war',
    'sf': 'sf',
    '다큐멘터리': 'documentary',
    '서부': 'western'
}

def translate_genres(genre_list, genre_dict):
  return [genre_dict.get(g, g).lower() for g in genre_list]

movies['genres'] = movies['genres'].apply(lambda x: translate_genres(x, genre_translate))

# 빠뜨린 장르 있는지 확인
all_genres = set([g for genre_list in movies['genres'] for g in genre_list])
korean_genres = [g for g in all_genres if re.search(r'[가-힣]', g)]

print(korean_genres)

['tv 영화']


In [8]:
# 장르 매핑(영화 장르 - 게임 장르)
movie_to_game_genre_map = {
    'comedy': ['casual', 'indie'],
    'romance': ['casual', 'simulation'],
    'drama': ['adventure', 'story-rich'],
    'family': ['family', 'casual'],
    'music': ['music', 'rhythm'],
    'horror': ['action', 'adventure'],
    'fantasy': ['rpg', 'adventure'],
    'action': ['action', 'shooter'],
    'adventure': ['adventure', 'rpg'],
    'animation': ['indie', 'casual'],
    'tv movie': ['casual'],
    'crime': ['strategy', 'action'],
    'thriller': ['action', 'strategy'],
    'history': ['strategy', 'simulation'],
    'mystery': ['puzzle', 'strategy'],
    'war':['strategy', 'action'],
    'sf': ['shooter', 'action'],
    'documentary': ['educational'],
    'western': ['action', 'adventure']
}

def map_movie_to_game_genres(genres, mapping_dict):
  mapped = []
  for g in genres:
    mapped.extend(mapping_dict.get(g, ['indie']))
  return list(set(mapped))

In [13]:
# 투자 대비 수익률 계산 : (revenue-budget)/budget
# 일단 코드 짜놓긴 했는데 결측치가 많아서 쓸 수 있을지 모르겠음
movies['budget'] = movies['budget'].fillna(0)
movies['revenue'] = movies['revenue'].fillna(0)
movies['roi'] = movies.apply(lambda x: (x['revenue']-x['budget'])/x['budget'] if x['budget'] > 0 else 0, axis=1)

In [14]:
# 수치형 컬럼 정규화
num_cols = ['popularity', 'vote_average', 'vote_count', 'runtime', 'roi']
for col in num_cols:
  movies[col] = movies[col].fillna(0)

# 로그 변환 (vote_count 분포가 극단적으로 불균형함)
movies['log_vote_count'] = np.log1p(movies['vote_count'])

# MinMax 정규화
scaler = MinMaxScaler()
movies[['popularity', 'vote_average', 'log_vote_count', 'runtime', 'roi']] = scaler.fit_transform(
    movies[['popularity', 'vote_average', 'log_vote_count', 'runtime', 'roi']]
)

In [9]:
# 최종 저장할 컬럼 선택
# clean_movies = movies[[
#    'tmdb_id', 'title', 'original_title', 'release_date', 'overview', 'genres',
#    'keywords', 'popularity', 'vote_average', 'log_vote_count', 'runtime', 'roi'
# ]]

clean_movies = movies[[
    'tmdb_id', 'title', 'original_title', 'release_date', 'overview', 'genres',
    'keywords', 'popularity', 'vote_average', 'vote_count', 'runtime'
]]

In [10]:
# 저장
clean_movies.to_csv('/content/drive/MyDrive/BigData/Project/cleaned_movies_data.csv', index=False)

In [20]:
cleaned_movies = pd.read_csv('/content/drive/MyDrive/BigData/Project/cleaned_movies_data.csv')
print(cleaned_movies.shape)
cleaned_movies.head(5)

(5000, 12)


,tmdb_id,title,original_title,release_date,overview,genres,keywords,popularity,vote_average,log_vote_count,runtime,roi
0,617126,판타스틱 4: 새로운 출발,The Fantastic 4: First Steps,2025-07-22,1960년대 전 세계적인 관심 속 우주로 떠난 4명의 엘리트 우주비행사 리드 리처드...,"['sf', 'adventure']","['superhero', 'based on comic', 'end of the wo...",1.000000,0.7189,0.693074,0.246220,0.002604
1,1311031,극장판 귀멸의 칼날: 무한성편,劇場版「鬼滅の刃」無限城編 第一章 猗窩座再来,2025-07-18,혈귀로 변해버린 여동생 네즈코를 인간으로 되돌리기 위해 혈귀를 사냥하는 조직인 귀살...,"['animation', 'action', 'fantasy', 'thriller']","['supernatural', 'sequel', 'based on manga', '...",0.839427,0.7800,0.557655,0.334773,0.027938
2,755898,우주전쟁,War of the Worlds,2025-07-29,전설적인 동명 소설을 새롭게 재해석한 이번 작품은 거대한 침공의 서막을 알린다 에바...,"['sf', 'thriller']","['government', 'attack', 'privacy', 'surveilla...",0.659414,0.4400,0.603247,0.313175,0.001000
3,987400,Batman Azteca: Choque de imperios,Batman Azteca: Choque de imperios,2025-09-18,in the time of the aztec empire tragedy strike...,"['animation', 'action', 'adventure', 'fantasy']","['death of father', 'based on comic', 'colonis...",0.456307,0.8072,0.402888,0.192225,0.001000
4,1038392,컨저링: 마지막 의식,The Conjuring: Last Rites,2025-09-03,1986년 펜실베니아 자신의 집에 사악한 존재가 들어왔다고 주장하는 스멀 일가를 조...,['horror'],"['pennsylvania, usa', 'supernatural', 'exorcis...",0.387290,0.6600,0.575033,0.293737,0.007332
